# Model V2 (Smaller Unit) 

One of the major feedback I received was that the architecture unit is likely too large. In this variation, I'll test using a smaller unit size. 

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.style.use('fivethirtyeight')
%matplotlib inline

from tensorflow.keras.applications import NASNetLarge 
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Model, model_from_json, load_model
from tensorflow.keras.layers import Input, Dropout, Dense, Embedding, LSTM, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

import pickle
import os
from timeit import default_timer as timer

import string
import re

In [ ]:
# loading data
descriptions = load_files('PKL/full_descriptions.pkl')
features = load_files('PKL/full_features.pkl')
train_list_full = load_files('PKL/train_list_full.pkl')
val_list_full = load_files('PKL/val_list_full.pkl')
test_list = load_files('PKL/test_list.pkl')
test_list_art = load_files('PKL/test_list_art.pkl')